In [1]:
"""
1
46  different injections (0 ~ 9.1, step=0.2)
500 trials for each n_inj (run on 1 core)
60 s/trial
= 12k cpu hr = 1600 hivecpu * 12 hr

tws_in_second    = [  10,     25,    50,   100,   250,   500][::-1] 
"""

'\n1\n46  different injections (0 ~ 9.1, step=0.2)\n500 trials for each n_inj (run on 1 core)\n60 s/trial\n= 12k cpu hr = 1600 hivecpu * 12 hr\n\ntws_in_second    = [  10,     25,    50,   100,   250,   500][::-1] \n'

In [2]:
import sys
import os
import glob
import numpy as np
import pandas as pd
sys.path.append('../../../')
from greco_grb.scripts.utils import *

In [3]:
from greco_grb.scripts import SETTING
paths = SETTING.PATH(osg=False)
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [4]:
### create arguments arrays ---> 13.txt
# 0: grb_name
# 1: n_inj
# 2: n_trials
# 3: ncpu
# 4: batchIndex
# 5: use_poisson
# 6: gamma

# Check 1 out of 2

In [5]:
####### Check before running ########
script_path    = '/scratch/cjchen/submit/13.sub'
## [::-1] reverse order
grb_names      = ['GRB160804D','GRB190321A','GRB170915A']  # top-2, top-4, top-5  
                # ['GRB140807A', 'GRB160802A', 'GRB190114C'] # top-1, top-3, comparision GRB
n_injs         = np.round(np.r_[0:9.1:0.2], 2)
n_trials       = 500
ncpu           = 4
batchIndex     = 0
use_poisson    = 1
gammas = [2.5, 2.0]
#######################################

In [6]:
contents = ""
for grb_name in grb_names:       
    for n_inj in n_injs:
        for gamma in gammas:
            contents += "{}\t".format(grb_name)
            contents += "{}\t".format(n_inj)
            contents += "{}\t".format(n_trials)
            contents += "{}\t".format(ncpu)
            contents += "{}\t".format(batchIndex)
            contents += "{}\t".format(use_poisson)
            contents += "{}\n".format(gamma)

with open("13.txt", 'w') as writer:
    writer.write(contents)
    

In [7]:
print(contents)

GRB160804D	0.0	500	4	0	1	2.5
GRB160804D	0.0	500	4	0	1	2.0
GRB160804D	0.2	500	4	0	1	2.5
GRB160804D	0.2	500	4	0	1	2.0
GRB160804D	0.4	500	4	0	1	2.5
GRB160804D	0.4	500	4	0	1	2.0
GRB160804D	0.6	500	4	0	1	2.5
GRB160804D	0.6	500	4	0	1	2.0
GRB160804D	0.8	500	4	0	1	2.5
GRB160804D	0.8	500	4	0	1	2.0
GRB160804D	1.0	500	4	0	1	2.5
GRB160804D	1.0	500	4	0	1	2.0
GRB160804D	1.2	500	4	0	1	2.5
GRB160804D	1.2	500	4	0	1	2.0
GRB160804D	1.4	500	4	0	1	2.5
GRB160804D	1.4	500	4	0	1	2.0
GRB160804D	1.6	500	4	0	1	2.5
GRB160804D	1.6	500	4	0	1	2.0
GRB160804D	1.8	500	4	0	1	2.5
GRB160804D	1.8	500	4	0	1	2.0
GRB160804D	2.0	500	4	0	1	2.5
GRB160804D	2.0	500	4	0	1	2.0
GRB160804D	2.2	500	4	0	1	2.5
GRB160804D	2.2	500	4	0	1	2.0
GRB160804D	2.4	500	4	0	1	2.5
GRB160804D	2.4	500	4	0	1	2.0
GRB160804D	2.6	500	4	0	1	2.5
GRB160804D	2.6	500	4	0	1	2.0
GRB160804D	2.8	500	4	0	1	2.5
GRB160804D	2.8	500	4	0	1	2.0
GRB160804D	3.0	500	4	0	1	2.5
GRB160804D	3.0	500	4	0	1	2.0
GRB160804D	3.2	500	4	0	1	2.5
GRB160804D	3.2	500	4	0	1	2.0
GRB160804D	3.4

In [8]:
print("Total number of jobs: {}\n".format(contents.count('\n')))

Total number of jobs: 276



In [9]:
### create pbs file ---> 13.pbs

# Check 2 out of 2

In [10]:
nameofjob  = "13"
ntotal     = 276           # copy from the above 2nd cell "Total number of jobs: xxx"
inputtxt   = "13.txt"      # copy from the above 3rd cell
outdir     = ensure_dir("/storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out")
walltimehr = 100 # 2hr is needed; 8 hrs for safe  # in Hour

In [11]:
contents = ""
contents += "#PBS -N {}\n".format(nameofjob)
contents += "#PBS -l nodes=1:ppn={}\n".format(4)           # number of nodes and cores per node required
contents += "#PBS -l pmem=4gb\n"                             # memory per core
contents += "#PBS -l walltime={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#PBS -q hive\n"
contents += "#PBS -j oe\n"
contents += "#PBS -o {}\n".format(outdir+"/13.out")
contents += "#PBS -t 1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "arg2=${paramsArray[2]}\n"
contents += "arg3=${paramsArray[3]}\n"
contents += "arg4=${paramsArray[4]}\n"
contents += "arg5=${paramsArray[5]}\n"
contents += "arg6=${paramsArray[6]}\n"
contents += "python 13_unblinding_upper_limit.py "

contents += "--grb_name $arg0 "
contents += "--n_inj $arg1 "
contents += "--n_trials $arg2 "
contents += "--ncpu $arg3 "
contents += "--batchIndex $arg4 "
contents += "--use_poisson $arg5 "
contents += "--gamma $arg6 "
print(contents)
with open("13.pbs", 'w') as writer:
    writer.write(contents)

#PBS -N 13
#PBS -l nodes=1:ppn=4
#PBS -l pmem=4gb
#PBS -l walltime=100:00:00
#PBS -q hive
#PBS -j oe
#PBS -o /storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out/13.out
#PBS -t 1-276%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/13.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
arg2=${paramsArray[2]}
arg3=${paramsArray[3]}
arg4=${paramsArray[4]}
arg5=${paramsArray[5]}
arg6=${paramsArray[6]}
python 13_unblinding_upper_limit.py --grb_name $arg0 --n_inj $arg1 --n_trials $arg2 --ncpu $arg3 --batchIndex $arg4 --use_poisson $arg5 --gamma $arg6 
